<a href="https://colab.research.google.com/github/gnakspandana/gnakspandana/blob/main/Fetch_the_recent_mail_in_gmail_gmail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install imapclient pyzmail36 openai langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 kB 4.4 MB/s eta 0:00:00


In [18]:
EMAIL_USER = "a.g.spandana@gmail.com"
APP_PASSWORD = "***********"

In [19]:
import imaplib
import email

def read_latest_email():
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(EMAIL_USER, APP_PASSWORD)
    mail.select("inbox")

    status, messages = mail.search(None, "ALL")
    email_ids = messages[0].split()
    latest_email_id = email_ids[-1]

    status, msg = mail.fetch(latest_email_id, "(RFC822)")
    raw_email = msg[0][1]
    msg = email.message_from_bytes(raw_email)

    subject = msg["Subject"]
    sender = msg["From"]

    # Extract body
    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == "text/plain":
                body = part.get_payload(decode=True).decode()
    else:
        body = msg.get_payload(decode=True).decode()

    return subject, sender, body


In [20]:
import smtplib
from email.mime.text import MIMEText

def send_email(to, subject, body):
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = EMAIL_USER
    msg["To"] = to

    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(EMAIL_USER, APP_PASSWORD)
    server.send_message(msg)
    server.quit()


In [21]:
from openai import OpenAI
import getpass

client = OpenAI(api_key=getpass.getpass("Enter your OpenAI API Key: "))

def ask_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message["content"]


Enter your OpenAI API Key: ··········


In [22]:
def gmail_chatbot():
    print("Welcome to Gmail AI Assistant!")
    print("Commands: read, reply, summarize, send, exit\n")

    while True:
        cmd = input("Enter command: ").lower()

        if cmd == "read":
            subject, sender, body = read_latest_email()
            print("\n📨 Latest Email:")
            print("From:", sender)
            print("Subject:", subject)
            print("Body:\n", body)

        elif cmd == "summarize":
            subject, sender, body = read_latest_email()
            summary = ask_gpt(f"Summarize this email:\n\n{body}")
            print("\n🔎 Summary:\n", summary)

        elif cmd == "reply":
            subject, sender, body = read_latest_email()
            reply = ask_gpt(f"Write a reply to this email:\n\n{body}")
            print("\n✍️ Suggested Reply:\n", reply)

            send = input("Send reply? (yes/no): ")
            if send.lower() == "yes":
                send_email(sender, "Re: " + subject, reply)
                print("Reply sent!")

        elif cmd == "send":
            to = input("To: ")
            subject = input("Subject: ")
            body = input("Body: ")
            send_email(to, subject, body)
            print("Email sent!")

        elif cmd == "exit":
            print("Goodbye!")
            break

        else:
            print("Unknown command. Try again.")


In [23]:
gmail_chatbot()


Welcome to Gmail AI Assistant!
Commands: read, reply, summarize, send, exit

Enter command: read

📨 Latest Email:
From: Google <no-reply@accounts.google.com>
Subject: Security alert
Body:
 [image: Google]
App password created to sign in to your account


a.g.spandana@gmail.com
If you didn't generate this password for Mail, someone might be using your
account. Check and secure your account now.
Check activity
<https://accounts.google.com/AccountChooser?Email=a.g.spandana@gmail.com&continue=https://myaccount.google.com/alert/nt/1763642038358?rfn%3D20%26rfnc%3D1%26eid%3D8739283822934867792%26et%3D0>
You can also see security activity at
https://myaccount.google.com/notifications
You received this email to let you know about important changes to your
Google Account and services.
© 2025 Google Ireland Ltd., Gordon House, Barrow Street, Dublin 4, Ireland

Enter command: sort
Unknown command. Try again.
Enter command: exit
Goodbye!
